In [7]:
#!/usr/bin/env python  
# _*- coding: utf-8 -*-
# Author: Li Zhu < zhulipresent@gmail.com>
# modified by Hanyu Liu <lhy@calypso.cn> at Dec 13th, 2014
import os
import glob
import re
def pressure(caldir):
    info = os.popen('''grep external ''' + caldir + '''/OUTCAR | tail -n 1''').read().split()
    infokb = os.popen('''grep 'in kB' ''' + caldir + '''/OUTCAR | tail -n 1''').read().split()
    try:
        external = float(info[3])
        pullay = float(info[8])
#        print float(infokb[2]),float(infokb[3]),float(infokb[4])
        ex_x = float(infokb[2])-float(pullay)
        ex_y = float(infokb[3])-float(pullay)
        ex_z = float(infokb[4])-float(pullay)
#        print ex_x,ex_y,ex_z
    except:
        return False
    f = open(caldir + '/CONTCAR')
    for i in range(0, 6):
        line = f.readline()
    try:
        natom = sum(map(int, line.split()))
    except:
        line = f.readline()
        natom = sum(map(int, line.split()))
    f.close()
    f = open(caldir + '/OUTCAR')
    numi=[]
    fff=[]
    for line in f:
      fff.append(line)
    for i in range(len(fff)):
      if 'TOTAL-FORCE' in fff[i]:
        numi.append(i)
    f.close()
    atomf = []
    latom=True
    for i in range(int(natom)):
      temp = re.findall(r'[-]?\d+\.\d+', fff[i+numi[-1]+int(2)].strip('\n'))
      print(temp)
      length = len(temp)
      if length < 6:
          for i in range(length, 6):
              temp.append(0)
      temp = list(map(float, temp))
      atomf.append(temp)
    for i in range(int(natom)):
       if abs(atomf[i][3]) < 0.3 and abs(atomf[i][4]) < 0.3 and abs(atomf[i][5]) < 0.3:
          latom=True
       else:
          latom=False
          return False
#      print atomf[i][3],atomf[i][4],atomf[i][5]
    lstress=True 
    if abs(pullay - 0.0) < 1.:
#        if abs(external) < 20.: 
        if abs(ex_x) < 60. and abs(ex_y) < 60. and abs(ex_z) < 60. and abs(external) < 20.:
            return True
        else:
            return False
    elif pullay < 1000.:
#        if abs(external) < 50.:
        if abs(ex_x) < 150. and abs(ex_y) < 150. and abs(ex_z) < 150. and abs(external) < 50.:
            return True
        else:
            return False
    elif pullay < 10000.:
#        if abs(external) < 100.:
        if abs(ex_x) < 300. and abs(ex_y) < 300. and abs(ex_z) < 300. and abs(external) < 100.:
            return True
        else:
            return False
    else:
#        if abs(external) < 500.:
        if abs(ex_x) < 1500. and abs(ex_y) < 1500. and abs(ex_z) < 1500. and abs(external) < 500.:
            return True
        else:
            return False
def outEnergy(caldir):
    f = open(caldir + '/CONTCAR')
    for i in range(0, 6):
        line = f.readline()
    try:
        natom = sum(map(int, line.split()))
    except:
        line = f.readline()
        natom = sum(map(int, line.split()))
    f.close()
    osawk = '''awk '/enthalpy is  TOTEN    =/ {print $5}' ''' + caldir + '''/OUTCAR | tail -1'''
    b = os.popen(osawk).read() 
    a = b[0:-1]
    try:
        e = float(a) / natom
    except:
        e = 610612509
    return e
def hardness():
    try:
        f = open('hardnes.dat')
    except:
        return 0
    try:
        b = f.readline()
    except:
        return 0
    try:
        h = float(b)
    except:
        return 0
    return h
def enthalpy(caldir):
    if pressure(caldir):
        e = outEnergy(caldir)
    else:
        e = 610612509
    # h = hardness()
    return e
    
def recordout():
    f = open('input.dat', 'r')
    indata = {}
    for line in f:
        if '=' in line:
            if line.strip()[0] != '#':
                litem = line.split('=')
                indata[litem[0].strip()] = litem[1].strip()
    f.close()
    
    try:
        RO = indata['RecordOUTCAR'][0]
        if RO.upper() == 'T': 
            record = True
        else:
            record = False   
    except:
        # print 'exce'
        record = False
    return record
if __name__ == '__main__':
    print(enthalpy('./'))
    if recordout():
        noo = len(glob.glob('OUTCAR*'))
        os.system('cp OUTCAR OUTCAR_' + str(noo))
        os.system('bzip2 OUTCAR_' + str(noo))
        OS = len(glob.glob('OSZICAR*'))
        os.system('cp OSZICAR OSZICAR_' + str(OS))
        os.system('bzip2 OSZICAR_' + str(OS))
        OU = len(glob.glob('CONTCAR*'))
        os.system('cp CONTCAR CONTCAR_' + str(OU))
        os.system('bzip2 CONTCAR_' + str(OU))



['2.54925', '1.33048', '0.00076', '1.653730', '-0.954809', '-194.425926']
['-0.16925', '2.90000', '3.90164', '-2.819371', '1.627755', '-67.619473']
['2.54371', '1.33368', '0.93713', '-2.266256', '1.308490', '468.269088']
['2.53764', '1.33718', '3.36164', '54990.881424', '-31748.935373', '469285.917413']
['-0.13677', '2.88126', '1.91374', '-0.800787', '0.462338', '24.339074']
['4.12485', '-0.14117', '0.00268', '-3.228656', '0.046174', '0.271095']
['2.95372', '3.21759', '0.02152', '1.586924', '-1.951122', '-0.024679']
['0.52728', '0.81055', '0.02191', '1.094787', '1.878088', '-0.161904']
['4.61153', '0.70179', '0.00268', '-1.654316', '2.773011', '0.271095']
['1.11720', '0.03665', '0.02152', '2.483183', '-0.398756', '-0.024679']
['1.98854', '3.34152', '0.02191', '-1.079078', '-1.887157', '-0.161904']
['-0.93826', '3.34399', '2.61433', '-11.849607', '6.841375', '5.601476']
['-0.11409', '1.92848', '2.58210', '0.469311', '-13.872749', '5.755313']
['0.69970', '3.33800', '2.58210', '12.248810'

FileNotFoundError: [Errno 2] No such file or directory: 'input.dat'